In [61]:
import sys

sys.path.append("../src")

import altair as alt
alt.renderers.enable("jupyter", offline=True)
alt.data_transformers.disable_max_rows()

import gc
import os
import polars as pl
import polars.selectors as cs
import warnings
from sklearn.model_selection import GroupKFold
from dataclasses import asdict

import lightgbm as lgb
from dotenv import load_dotenv

import wandb
from pathlib import Path

from config import cfg
from data.data_class import Dfs
from data.feature_eng import preprocess, feature_eng
from data.data_process import add_fold, get_Xy
from models.lgb import train_model

# warnings.filterwarnings("ignore")
# warnings.simplefilter("ignore")

cfg.train_path = Path("../data/train.csv")
cfg.test_path = Path("../data/test.csv")
cfg.pltpd_path = Path("../data/podcast_dataset.csv")


df_test = pl.read_csv(cfg.test_path)

df_train = pl.read_csv(cfg.train_path)
df_train = df_train.filter(pl.col("Number_of_Ads").is_not_null())

df_train = df_train.drop("id")
df_train = preprocess(df_train)
df_train = feature_eng(df_train, df_train)

df = df_train.clone()
df

Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Num,Episode_Length_minutes_NaN,Guest_Popularity_percentage_NaN,Day_sin,Day_cos,Day_sin2,Day_cos2,Time_sin,Time_cos,Time_sin2,Time_cos2,Length_per_Ads,Length_per_Host,Length_per_Guest,ELen_Int,ELen_Dec,HPperc_Int,HPperc_Dec,Is_Positive_Sentiment,Sentiment_Multiplier,Episode_Length_squared,Episode_Length_squared2,Long_Term_Cycle_Sin,Long_Term_Cycle_Cos,Expected_Listening_Time_Sentiment,Episode_Length_minutes_Host_Popularity_percentage_max,Episode_Length_minutes_Host_Popularity_percentage_min,…,Host_Popularity_percentage_sq_ELen_Int,Host_Popularity_percentage_ELen_Int_sq,Host_Popularity_percentage_ELen_Dec_max,Host_Popularity_percentage_ELen_Dec_min,Host_Popularity_percentage_ELen_Dec_range,Host_Popularity_percentage_sq_ELen_Dec,Host_Popularity_percentage_ELen_Dec_sq,Guest_Popularity_percentage_Number_of_Ads_max,Guest_Popularity_percentage_Number_of_Ads_min,Guest_Popularity_percentage_Number_of_Ads_range,Guest_Popularity_percentage_sq_Number_of_Ads,Guest_Popularity_percentage_Number_of_Ads_sq,Guest_Popularity_percentage_ELen_Int_max,Guest_Popularity_percentage_ELen_Int_min,Guest_Popularity_percentage_ELen_Int_range,Guest_Popularity_percentage_sq_ELen_Int,Guest_Popularity_percentage_ELen_Int_sq,Guest_Popularity_percentage_ELen_Dec_max,Guest_Popularity_percentage_ELen_Dec_min,Guest_Popularity_percentage_ELen_Dec_range,Guest_Popularity_percentage_sq_ELen_Dec,Guest_Popularity_percentage_ELen_Dec_sq,Number_of_Ads_ELen_Int_max,Number_of_Ads_ELen_Int_min,Number_of_Ads_ELen_Int_range,Number_of_Ads_sq_ELen_Int,Number_of_Ads_ELen_Int_sq,Number_of_Ads_ELen_Dec_max,Number_of_Ads_ELen_Dec_min,Number_of_Ads_ELen_Dec_range,Number_of_Ads_sq_ELen_Dec,Number_of_Ads_ELen_Dec_sq,ELen_Int_ELen_Dec_max,ELen_Int_ELen_Dec_min,ELen_Int_ELen_Dec_range,ELen_Int_sq_ELen_Dec,ELen_Int_ELen_Dec_sq
cat,f32,cat,f32,cat,cat,f32,f32,cat,f32,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,f32,f32,f32,f64,f64,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""0""",63.84,"""0""",74.809998,"""3""","""21""",53.580002,0.0,"""2""",31.419979,"""98""","""true""","""true""",0.433884,-0.900969,-0.781831,0.62349,1.0,-0.000002,-1.0,6.6361e-7,63.84,0.842105,1.169659,63.0,0.84,74.0,0.809998,1,0.75,4075.545654,260182.828125,-0.125333,0.992115,47.880001,74.809998,63.84,…,352581.75,296920.875,74.809998,0.84,73.970001,4701.09082,52.785954,53.580002,0.0,53.580002,0.0,0.0,63.0,53.580002,9.419998,180861.453125,212659.03125,53.580002,0.84,52.740002,2411.486328,37.806065,63.0,0.0,63.0,0.0,0.0,0.84,0.0,0.84,0.0,0.0,63.0,0.84,62.16,3333.960693,44.452816
"""1""",119.800003,"""1""",66.949997,"""5""","""14""",75.949997,2.0,"""0""",88.012413,"""26""","""false""","""false""",-0.974928,-0.222521,0.433884,-0.900969,-0.000001,-1.0,0.866026,0.5,39.933334,1.763061,1.556855,119.0,0.800003,66.0,0.949997,0,0.717,14352.041016,1719374.5,0.998027,-0.062791,85.896606,119.800003,66.949997,…,533393.9375,948078.9375,66.949997,0.800003,66.149994,3585.855469,42.848324,75.949997,2.0,73.949997,11536.803711,303.799988,119.0,75.949997,43.050003,686439.8125,1.0755e6,75.949997,0.800003,75.149994,4614.739258,48.608368,119.0,2.0,117.0,476.0,28322.0,2.0,0.800003,1.199997,3.200012,1.28001,119.0,0.800003,118.199997,11328.842773,76.160583
"""2""",73.900002,"""2""",69.970001,"""1""","""17""",8.97,0.0,"""0""",44.925308,"""16""","""false""","""false""",0.781832,0.62349,0.974928,-0.222521,1.0,-0.000001,0.5,-0.866025,73.900002,1.041285,7.412237,73.0,0.900002,69.0,0.970001,0,0.717,5461.210449,403583.46875,0.844328,0.535827,52.986301,73.900002,69.970001,…,357393.5,372870.125,69.970001,0.900002,69.07,4406.228516,56.675892,8.97,0.0,8.97,0.0,0.0,73.0,8.97,64.029999,5873.645996,47801.132812,8.97,0.900002,8

In [62]:
from itertools import combinations


# cols = df_train.columns
# cols = [col for col in cols if col not in ["id", "Listening_Time_minutes", "fold"]]

cols = ["Podcast_Name", "Episode_Length_minutes", "Genre", "Host_Popularity_percentage", "Publication_Day", "Publication_Time", "Guest_Popularity_percentage", "Number_of_Ads", "Episode_Sentiment", "Episode_Num", "Episode_Length_minutes_NaN", "Guest_Popularity_percentage_NaN", "Day_sin", "Day_cos", "Day_sin2", "Day_cos2", "Time_sin", "Time_cos", "Time_sin2", "Time_cos2", "Length_per_Ads", "Length_per_Host", "Length_per_Guest", "ELen_Int", "ELen_Dec", "HPperc_Int", "HPperc_Dec", "Is_Positive_Sentiment", "Sentiment_Multiplier", "Long_Term_Cycle_Sin", "Long_Term_Cycle_Cos"]

combs = []
combs += list(combinations(cols, 1))
combs += list(combinations(cols, 2))
combs += list(combinations(cols, 3))
# combs += list(combinations(cols, 4))

std_distributes = []
for comb in combs:
    concat_expr = pl.col(comb[0]).cast(pl.Utf8)
    for col_name in comb[1:]:
        concat_expr = concat_expr + "_" + pl.col(col_name).cast(pl.Utf8)
    df_train = df_train.with_columns(
        concat_expr.alias("group").cast(pl.Categorical)
    )

    df_group = df_train.group_by("group").agg(
        pl.col("Listening_Time_minutes").count().alias("count"),
        pl.col("Listening_Time_minutes").std().alias("std_listening_time"),
        pl.col("Listening_Time_minutes").mean().alias("mean_listening_time"),
        pl.col("Episode_Length_minutes").mean().alias("mean_episode_length"),
        
    )
    df_group = df_group.filter(~pl.col("std_listening_time").is_null())
    if len(df_group) == 0:
        print(f"Skip {comb}")
        continue

    std_distributes.append({
        "group": "-".join(comb),
        "len": len(df_group),
        "mean_count": df_group["count"].mean(),
        "mean_std_listening_time": df_group["std_listening_time"].mean(),
    })

std_distributes = pl.DataFrame(std_distributes)
std_distributes = std_distributes.sort("mean_std_listening_time")
std_distributes.write_csv("../data/std_distributes.csv")
std_distributes

group,len,mean_count,mean_std_listening_time
str,i64,f64,f64
"""Podcast_Name-Episode_Num-Lengt…",9082,2.229905,0.438956
"""Podcast_Name-Length_per_Host-L…",9148,2.22923,0.502984
"""Podcast_Name-Length_per_Host-L…",9133,2.228731,0.519117
"""Publication_Day-Episode_Num-Le…",14217,2.293874,1.419231
"""Episode_Num-Day_sin-Length_per…",14217,2.293874,1.419231
…,…,…,…
"""Day_cos-Time_sin-Sentiment_Mul…",42,17857.119048,27.338677
"""Day_sin2-Time_sin-Is_Positive_…",42,17857.119048,27.338677
"""Day_sin2-Time_sin-Sentiment_Mu…",42,17857.119048,27.338677


In [ ]:
std_distributes.plot.point(
    x="len",
    y="mean_std_listening_time",
    color="group",
).properties(
    width=600,
    height=400,
)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': 'da…

In [50]:
import altair as alt
import polars as pl


def plot_highlighted_groups(highlight_groups):
    global std_distributes

    plot_df = std_distributes.with_columns(
        pl.when(pl.col("group").is_in(highlight_groups))
        .then(True)
        .otherwise(False)
        .alias("highlighted")
    )

    chart = alt.Chart(plot_df).mark_point().encode(
        x='len',
        y='mean_std_listening_time',
        color='group',
        size=alt.condition(
            alt.datum.highlighted == True,
            alt.value(50),
            alt.value(10)
        ),
        opacity=alt.condition(
            alt.datum.highlighted == True,
            alt.value(1.0),
            alt.value(0.3)
        ),
        tooltip=['group', 'len', 'mean_std_listening_time']
    ).properties(
        width=600,
        height=400,
        title='Highlighted Groups in Std Distributes'
    )

    print("Highlighted Groups:", highlight_groups)
    chart.show()


In [51]:
highlight_groups = std_distributes["group"].to_list()[:20]
plot_highlighted_groups(highlight_groups)

Highlighted Groups: ['Episode_Num-Length_per_Guest', 'Length_per_Guest-HPperc_Dec', 'Length_per_Guest-HPperc_Int', 'Episode_Num-Length_per_Host', 'Length_per_Guest-Long_Term_Cycle_Sin', 'Length_per_Guest-Long_Term_Cycle_Cos', 'Length_per_Host-Long_Term_Cycle_Cos', 'Length_per_Host-Long_Term_Cycle_Sin', 'Episode_Length_minutes-Host_Popularity_percentage', 'Episode_Length_minutes-Length_per_Host', 'Host_Popularity_percentage-Length_per_Host', 'Length_per_Host-ELen_Int', 'Length_per_Host-HPperc_Int', 'Length_per_Host-HPperc_Dec', 'Length_per_Host-ELen_Dec', 'Length_per_Host-Length_per_Guest', 'Host_Popularity_percentage-Length_per_Guest', 'Podcast_Name-Length_per_Guest', 'Guest_Popularity_percentage-Length_per_Host', 'Episode_Length_minutes-HPperc_Dec']


JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': 'da…

In [ ]:
highlight_groups = std_distributes.sort("mean_std_listening_time", descending=True)["group"].to_list()[:20]
plot_highlighted_groups(highlight_groups)

Highlighted Groups: ['Time_sin-Is_Positive_Sentiment', 'Time_sin-Sentiment_Multiplier', 'Publication_Day-Is_Positive_Sentiment', 'Publication_Day-Sentiment_Multiplier', 'Day_sin-Is_Positive_Sentiment', 'Day_sin-Sentiment_Multiplier', 'Day_cos-Is_Positive_Sentiment', 'Day_cos-Sentiment_Multiplier', 'Day_sin2-Is_Positive_Sentiment', 'Day_sin2-Sentiment_Multiplier', 'Day_cos2-Is_Positive_Sentiment', 'Day_cos2-Sentiment_Multiplier', 'Genre-Is_Positive_Sentiment', 'Genre-Sentiment_Multiplier', 'Is_Positive_Sentiment', 'Sentiment_Multiplier', 'Is_Positive_Sentiment-Sentiment_Multiplier', 'Publication_Time-Is_Positive_Sentiment', 'Publication_Time-Sentiment_Multiplier', 'Time_sin2-Is_Positive_Sentiment']


JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': 'da…

In [ ]:
def qcut_least_std(df, n_bins=5, min_mean_std=12.0):
    with_bins = df.with_columns(
        pl.col("len").log10().alias("len_log")
    )
    with_bins = with_bins.with_columns(
        pl.col("len").qcut(n_bins).alias("len_bin")
    )
    print(with_bins["len_bin"].unique().to_list())

    result = with_bins.group_by("len_bin").agg(
        pl.col("group").filter((pl.col("mean_std_listening_time").min() < min_mean_std) & (pl.col("mean_std_listening_time") == pl.col("mean_std_listening_time").min()))
    ).sort("len_bin")
    result = result.filter(pl.col("group").len() > 0)

    least_groups = [item[0] for item in result["group"].to_list() if len(item) > 0]
    return least_groups

least_groups = qcut_least_std(std_distributes, n_bins=20)
print("Length of least groups:", len(least_groups))
plot_highlighted_groups(least_groups)

['(-inf, 4]', '(4, 7]', '(7, 12]', '(12, 16]', '(16, 28]', '(28, 59]', '(59, 172.25]', '(172.25, 273]', '(273, 367]', '(367, 592]', '(592, 810]', '(810, 2215]', '(2215, 11648]', '(11648, 22850.5]', '(22850.5, 35830.25]', '(35830.25, 41873]', '(41873, 50890.5]', '(50890.5, 63601]', '(63601, 113793.75]', '(113793.75, inf]']
Highlighted Groups: ['Episode_Length_minutes_NaN-Guest_Popularity_percentage_NaN', 'Number_of_Ads', 'Number_of_Ads-Episode_Length_minutes_NaN', 'Number_of_Ads-Time_cos', 'Genre-Episode_Length_minutes_NaN', 'Publication_Day-Number_of_Ads', 'ELen_Int', 'ELen_Int-Is_Positive_Sentiment', 'Episode_Sentiment-ELen_Int', 'ELen_Dec', 'Guest_Popularity_percentage_NaN-ELen_Dec', 'ELen_Dec-Is_Positive_Sentiment', 'Episode_Length_minutes', 'Length_per_Guest-HPperc_Dec', 'Episode_Num-Length_per_Guest', 'Episode_Length_minutes-Host_Popularity_percentage', 'Length_per_Host', 'Length_per_Ads-Length_per_Guest', 'Episode_Num-Length_per_Ads', 'Episode_Length_minutes-HPperc_Dec']


JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': 'da…